In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

from retinaface import RetinaFace
print("RetinaFace successfully imported!")


In [ ]:
import tensorflow as tf
print("GPUs Available:", tf.config.list_physical_devices('GPU'))
print("TensorFlow Version:", tf.__version__)
with tf.device('/GPU:0'):
    a = tf.constant([1.0, 2.0, 3.0])
    b = tf.constant([4.0, 5.0, 6.0])
    c = a * b
    print("TensorFlow GPU Test:", c)


In [1]:
import cv2
import tensorflow as tf
import threading
import time
from retinaface import RetinaFace
from queue import Queue

2025-03-08 13:59:30.723294: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-08 13:59:31.250897: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 13:59:31.498762: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 13:59:33.022634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [3]:
import threading
import time
import cv2
import tensorflow as tf
from queue import Queue
from retinaface import RetinaFace
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("[INFO] TensorFlow is using GPU:", tf.config.list_physical_devices('GPU'))
    except RuntimeError as e:
        print(e)

# Camera sources
CAMERA_SOURCES = {
    "camera1": "http://10.24.75.85:4747/video",
    "camera2": "http://10.24.78.21:4747/video",
}
frames = {name: Queue(maxsize=1) for name in CAMERA_SOURCES}
FRAME_SKIP = 2  

def detect_faces(frame):
    """Run RetinaFace on GPU and detect faces."""
    if frame is None or frame.size == 0:
        return None

    faces = RetinaFace.detect_faces(frame)  # RetinaFace uses MXNet by default

    if isinstance(faces, dict) and faces:
        for _, face in faces.items():
            x1, y1, x2, y2 = face["facial_area"]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    return frame

def capture_and_process_camera(camera_name, url):
    """Capture video and run RetinaFace detection."""
    cap = cv2.VideoCapture(url)
    if not cap.isOpened():
        print(f"[ERROR] Unable to open {camera_name}. Check IP and connectivity.")
        return

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print(f"[ERROR] Lost connection to {camera_name}")
            break
        frame = cv2.resize(frame, (640, 480))

        frame_count += 1
        if frame_count % FRAME_SKIP == 0:
            processed_frame = detect_faces(frame)
            if processed_frame is not None:
                try:
                    frames[camera_name].put_nowait(processed_frame)  # Non-blocking
                except:
                    pass  

    cap.release()

# Start threads for each camera
threads = []
for name, url in CAMERA_SOURCES.items():
    thread = threading.Thread(target=capture_and_process_camera, args=(name, url), daemon=True)
    thread.start()
    threads.append(thread)
    time.sleep(1)

# Display video streams
while True:
    for name in CAMERA_SOURCES.keys():
        try:
            frame = frames[name].get_nowait()  # Non-blocking
            cv2.imshow(f"Live Stream - {name}", frame)
        except:
            pass  # No frame available yet

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()


[INFO] TensorFlow is using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[ERROR] Unable to open camera1. Check IP and connectivity.
[ERROR] Lost connection to camera2


[http @ 0x7f86010207c0] Stream ends prematurely at 19058945, should be 18446744073709551615


[ERROR] Lost connection to camera1


[http @ 0x7f8608001cc0] Stream ends prematurely at 57317040, should be 18446744073709551615


In [6]:
import threading
import time
import cv2
import tensorflow as tf
import numpy as np
from queue import Queue
from retinaface import RetinaFace

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("[INFO] TensorFlow is using GPU:", tf.config.list_physical_devices('GPU'))
    except RuntimeError as e:
        print(e)

# Camera sources
CAMERA_SOURCES = {
    "camera1": "http://10.24.75.85:4747/video",
    ##"camera2": "http://10.24.78.21:4747/video",
}
frames = {name: Queue(maxsize=1) for name in CAMERA_SOURCES}
faces_queue = Queue(maxsize=10)  # Stores cropped face images

def detect_faces(frame):
    """Run RetinaFace (TensorFlow) for face detection."""
    if frame is None or frame.size == 0:
        return None

    faces = RetinaFace.detect_faces(frame)  # RetinaFace internally uses TensorFlow

    if isinstance(faces, dict) and faces:
        for _, face in faces.items():
            x1, y1, x2, y2 = face["facial_area"]
            cropped_face = frame[y1:y2, x1:x2]

            # Store cropped face for embedding extraction
            if cropped_face.size > 0:
                try:
                    faces_queue.put_nowait(cropped_face)
                except:
                    pass  # Skip if queue is full

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    return frame

def capture_and_process_camera(camera_name, url):
    """Capture video and process with RetinaFace."""
    cap = cv2.VideoCapture(url)
    if not cap.isOpened():
        print(f"[ERROR] Unable to open {camera_name}")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (640, 480))

        processed_frame = detect_faces(frame)

        try:
            frames[camera_name].put_nowait(processed_frame)
        except:
            pass  # Skip if queue is full

    cap.release()

# Start camera threads
for name, url in CAMERA_SOURCES.items():
    thread = threading.Thread(target=capture_and_process_camera, args=(name, url), daemon=True)
    thread.start()
    time.sleep(1)

# Display video
while True:
    for name in CAMERA_SOURCES.keys():
        try:
            frame = frames[name].get_nowait()
            cv2.imshow(f"Live Stream - {name}", frame)
        except:
            pass  

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()


[INFO] TensorFlow is using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


[http @ 0x7f8608014d80] Stream ends prematurely at 95921269, should be 18446744073709551615


/bin/bash: /home/student/anaconda3/envs/tensorflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ERROR: Could not find a version that satisfies the requirement mxnet-cu118 (from versions: none)
ERROR: No matching distribution found for mxnet-cu118
Note: you may need to restart the kernel to use updated packages.
